In [1]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors="tf"))

# This is new
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
labels = tf.convert_to_tensor([1, 1])
model.train_on_batch(batch, labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


13.421859741210938

In [1]:
!pip install datasets
# important import below due to non compatibility between tensorflow and keras models it is needed
!pip install keras==2.15.0
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.6 MB/s 

In [2]:
from datasets import load_dataset

In [3]:
raw_dataset=load_dataset("glue","mrpc")
raw_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
raw_dataset["train"]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [5]:
raw_dataset["test"]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 1725
})

In [6]:
raw_dataset["validation"]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 408
})

In [7]:
raw_train_dataset=raw_dataset["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [8]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [9]:
#preprocessing the sentence pairs
from transformers import AutoTokenizer
pre_model="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(pre_model)
def tokenize_function(examples):
  return tokenizer(examples["sentence1"],examples["sentence2"],padding="max_length",truncation=True,max_length=128)
# The tokenize_function is applied to a batch of examples at once. Instead of processing individual examples one by one, the function processes multiple examples simultaneously.
tokenized_datasets=raw_dataset.map(tokenize_function,batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [10]:
#Removing uneccessry columns from the dataset or renaming
tokenized_datasets=tokenized_datasets.remove_columns(["idx","sentence1","sentence2"])
tokenized_datasets=tokenized_datasets.rename_column("label","labels")
tokenized_datasets=tokenized_datasets.with_format("tensorflow")
tokenized_datasets["train"]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [11]:
small_train_dataset=tokenized_datasets["train"].select(range(100))

In [12]:
#for dynamic padding
from transformers import DataCollatorWithPadding
data_collator=DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="tf")

In [13]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:410: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [14]:
from transformers import TFAutoModelForSequenceClassification
model=TFAutoModelForSequenceClassification.from_pretrained(pre_model,num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#fine tunining the model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
#When you set from_logits=True in the loss function, you're telling the loss function that the model's output is raw logits and not probabilities. In this case, the loss function will internally apply the softmax function to the logits before computing the cross-entropy loss.
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)

Epoch 1/3
459/459 [==============================] - 161s 258ms/step - loss: 0.6821 - accuracy: 0.6200 - val_loss: 0.6866 - val_accuracy: 0.6838
Epoch 2/3
459/459 [==============================] - 111s 242ms/step - loss: 0.6889 - accuracy: 0.6268 - val_loss: 0.6881 - val_accuracy: 0.6838


In [15]:
from transformers import TFAutoModelForSequenceClassification
model1=TFAutoModelForSequenceClassification.from_pretrained(pre_model,num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
#using learning rate scheduler in tensorflow
from tensorflow.keras.optimizers.schedules import PolynomialDecay
batch_Size=8
epochs=3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps=len(tf_train_dataset)*epochs
learning_rate_scheduler=PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=num_train_steps,
)

from tensorflow.keras.optimizers import Adam


In [18]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model1.compile(
    optimizer=Adam(learning_rate=learning_rate_scheduler),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model1.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)

Epoch 1/3
459/459 [==============================] - 164s 250ms/step - loss: 0.6092 - accuracy: 0.6778 - val_loss: 0.5547 - val_accuracy: 0.7475
Epoch 2/3
459/459 [==============================] - 110s 240ms/step - loss: 0.4075 - accuracy: 0.8239 - val_loss: 0.4116 - val_accuracy: 0.8480
Epoch 3/3
459/459 [==============================] - 111s 241ms/step - loss: 0.1696 - accuracy: 0.9417 - val_loss: 0.3754 - val_accuracy: 0.8775


In [19]:
#as we can see abve the accuracy has increased a lot compared to before and loss also has significantly decreased

In [37]:
#prediction and matrices
import numpy as np
preds=model1.predict(tf_validation_dataset)["logits"]
class_preds=np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

51/51 [==============================] - 10s 102ms/step
(408, 2) (408,)


In [34]:
!pip install datasets evaluate transformers[sentencepiece]

In [39]:
import evaluate
metric=evaluate.load("glue","mrpc")
metric.compute(predictions=class_preds, references=raw_dataset["validation"]["label"])

{'accuracy': 0.8774509803921569, 'f1': 0.9140893470790378}